# Weather Dataset Preprocessing and feature Engineering

In [5]:
%AddJar file:///home/jovyan/work/apps/Emiasd-Flight-Data-Analysis.jar

Using cached version of Emiasd-Flight-Data-Analysis.jar


error: error while loading FlightDataLoader, class file '/tmp/toree-tmp-dir16172290280173686224/toree_add_jars/Emiasd-Flight-Data-Analysis.jar(com/flightdelay/data/loaders/FlightDataLoader.class)' has location not matching its contents: contains class FlightDataLoader


Using cached version of Emiasd-Flight-Data-Analysis.jar


In [6]:
import org.apache.spark.sql.SparkSession
import com.flightdelay.config.{AppConfiguration, ConfigurationLoader}
import com.flightdelay.data.loaders.FlightDataLoader

//Env Configuration
val args: Array[String] = Array("jupyter")

val spark = SparkSession.builder()
  .config(sc.getConf)
  .getOrCreate()

// Rendre la session Spark implicite
implicit val session = spark
implicit val configuration: AppConfiguration = ConfigurationLoader.loadConfiguration(args)

val rawFlightsPath = s"${configuration.common.output.basePath}/common/data/processed_flights.parquet"
val flightsDF = spark.read.parquet(rawFlightsPath)

args = Array(jupyter)
spark = org.apache.spark.sql.SparkSession@60cde8b2
session = org.apache.spark.sql.SparkSession@60cde8b2
configuration = AppConfiguration(local,CommonConfig(42,DataConfig(/home/jovyan/work/data,FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Flights/201201.csv),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Weather/201201hourly.txt),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/wban_airport_timezone.csv)),OutputConfig(/home/jovyan/work/output,FileConfig(/home/jovyan/work/output/data),FileCo...


AppConfiguration(local,CommonConfig(42,DataConfig(/home/jovyan/work/data,FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Flights/201201.csv),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Weather/201201hourly.txt),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/wban_airport_timezone.csv)),OutputConfig(/home/jovyan/work/output,FileConfig(/home/jovyan/work/output/data),FileCo...

In [7]:
flightsDF.count()

403884

In [8]:
flightsDF.printSchema

root
 |-- FL_DATE: date (nullable = true)
 |-- OP_CARRIER_AIRLINE_ID: integer (nullable = true)
 |-- OP_CARRIER_FL_NUM: integer (nullable = true)
 |-- ORIGIN_AIRPORT_ID: integer (nullable = true)
 |-- DEST_AIRPORT_ID: integer (nullable = true)
 |-- CRS_DEP_TIME: integer (nullable = true)
 |-- ARR_DELAY_NEW: double (nullable = true)
 |-- CRS_ELAPSED_TIME: double (nullable = true)
 |-- WEATHER_DELAY: double (nullable = true)
 |-- NAS_DELAY: double (nullable = true)
 |-- ORIGIN_WBAN: string (nullable = true)
 |-- ORIGIN_TIMEZONE: integer (nullable = true)
 |-- DEST_WBAN: string (nullable = true)
 |-- DEST_TIMEZONE: integer (nullable = true)
 |-- UTC_CRS_DEP_TIME: string (nullable = true)
 |-- UTC_ARR_TIME: string (nullable = true)
 |-- UTC_ARR_DATE: date (nullable = true)
 |-- CRS_ARR_TIME: string (nullable = true)
 |-- CRS_ARR_DATE: date (nullable = true)
 |-- feature_arrival_hour: integer (nullable = true)
 |-- feature_utc_arrival_hour: integer (nullable = true)
 |-- feature_utc_arrival

In [9]:
import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import org.apache.spark.ml.feature.{StringIndexer, OneHotEncoder, VectorAssembler}
import scala.util.Try

In [10]:
flightsDF.count()

403884

# feature_flight_year, feature_flight_month

In [11]:
import org.apache.spark.sql.functions._

val delaysByYearMonth =
  flightsDF
    .groupBy("feature_flight_year", "feature_flight_month")
    .agg(
      count(lit(1)).as("total_flights"),
      sum(col("label_is_delayed_15min").cast("long")).as("delayed_flights")
    )
    .orderBy(col("feature_flight_year"), col("feature_flight_month"))

delaysByYearMonth.show(10, truncate = false)

+-------------------+--------------------+-------------+---------------+
|feature_flight_year|feature_flight_month|total_flights|delayed_flights|
+-------------------+--------------------+-------------+---------------+
|2012               |1                   |10192        |4395           |
|2012               |2                   |9664         |3942           |
|2012               |3                   |11361        |5280           |
|2012               |4                   |10231        |4202           |
|2012               |5                   |10886        |4822           |
|2012               |6                   |11353        |5424           |
|2012               |7                   |11863        |5976           |
|2012               |8                   |11740        |5790           |
|2012               |9                   |10365        |4622           |
|2012               |10                  |11445        |5600           |
+-------------------+--------------------+---------

delaysByYearMonth = [feature_flight_year: int, feature_flight_month: int ... 2 more fields]


[feature_flight_year: int, feature_flight_month: int ... 2 more fields]

# feature_departure_hour_rounded

In [12]:
import org.apache.spark.sql.functions._

val delaysByYearMonth =
  flightsDF
    .groupBy("feature_departure_hour_rounded")
    .agg(
      count(lit(1)).as("total_flights"),
      sum(col("label_is_delayed_15min").cast("long")).as("delayed_flights")
    )
    .withColumn(
      "delay_rate_percent",
      round(col("delayed_flights") / col("total_flights") * 100, 2)
    )
    .orderBy(col("feature_departure_hour_rounded"))

delaysByYearMonth.show(10, truncate = false)

+------------------------------+-------------+---------------+------------------+
|feature_departure_hour_rounded|total_flights|delayed_flights|delay_rate_percent|
+------------------------------+-------------+---------------+------------------+
|0                             |1468         |724            |49.32             |
|100                           |504          |224            |44.44             |
|200                           |137          |65             |47.45             |
|300                           |60           |24             |40.0              |
|400                           |26           |7              |26.92             |
|500                           |936          |296            |31.62             |
|600                           |18043        |6134           |34.0              |
|700                           |23933        |8911           |37.23             |
|800                           |23988        |9866           |41.13             |
|900            

delaysByYearMonth = [feature_departure_hour_rounded: bigint, total_flights: bigint ... 2 more fields]


[feature_departure_hour_rounded: bigint, total_flights: bigint ... 2 more fields]

# feature_flight_day_of_week

In [13]:
import org.apache.spark.sql.functions._

val delaysByYearMonth =
  flightsDF
    .groupBy("feature_flight_day_of_week")
    .agg(
      count(lit(1)).as("total_flights"),
      sum(col("label_is_delayed_15min").cast("long")).as("delayed_flights")
    )
    .withColumn(
      "delay_rate_percent",
      round(col("delayed_flights") / col("total_flights") * 100, 2)
    )
    .orderBy(col("feature_flight_day_of_week"))

delaysByYearMonth.show(10, truncate = false)

+--------------------------+-------------+---------------+------------------+
|feature_flight_day_of_week|total_flights|delayed_flights|delay_rate_percent|
+--------------------------+-------------+---------------+------------------+
|1                         |56584        |27954          |49.4              |
|2                         |60808        |30740          |50.55             |
|3                         |57154        |27685          |48.44             |
|4                         |58830        |29197          |49.63             |
|5                         |61167        |32190          |52.63             |
|6                         |61262        |32256          |52.65             |
|7                         |48079        |22030          |45.82             |
+--------------------------+-------------+---------------+------------------+



delaysByYearMonth = [feature_flight_day_of_week: int, total_flights: bigint ... 2 more fields]


[feature_flight_day_of_week: int, total_flights: bigint ... 2 more fields]

# feature_flight_day_of_month

In [14]:
import org.apache.spark.sql.functions._

val delaysByYearMonth =
  flightsDF
    .groupBy("feature_flight_day_of_month")
    .agg(
      count(lit(1)).as("total_flights"),
      sum(col("label_is_delayed_15min").cast("long")).as("delayed_flights")
    )
    .withColumn(
      "delay_rate_percent",
      round(col("delayed_flights") / col("total_flights") * 100, 2)
    )
    .orderBy(col("feature_flight_day_of_month"))

delaysByYearMonth.show(10, truncate = false)

+---------------------------+-------------+---------------+------------------+
|feature_flight_day_of_month|total_flights|delayed_flights|delay_rate_percent|
+---------------------------+-------------+---------------+------------------+
|1                          |13061        |6689           |51.21             |
|2                          |13120        |6718           |51.2              |
|3                          |13333        |6649           |49.87             |
|4                          |12891        |6170           |47.86             |
|5                          |12879        |6014           |46.7              |
|6                          |12875        |6111           |47.46             |
|7                          |13041        |6325           |48.5              |
|8                          |13020        |6523           |50.1              |
|9                          |13197        |6613           |50.11             |
|10                         |13408        |6838     

delaysByYearMonth = [feature_flight_day_of_month: int, total_flights: bigint ... 2 more fields]


[feature_flight_day_of_month: int, total_flights: bigint ... 2 more fields]

# feature_flight_day_of_year

In [15]:
import org.apache.spark.sql.functions._

val delaysByYearMonth =
  flightsDF
    .groupBy("feature_flight_day_of_year")
    .agg(
      count(lit(1)).as("total_flights"),
      sum(col("label_is_delayed_15min").cast("long")).as("delayed_flights")
    )
    .withColumn(
      "delay_rate_percent",
      round(col("delayed_flights") / col("total_flights") * 100, 2)
    )
    .orderBy(col("delay_rate_percent").desc)

delaysByYearMonth.show(10, truncate = false)

+--------------------------+-------------+---------------+------------------+
|feature_flight_day_of_year|total_flights|delayed_flights|delay_rate_percent|
+--------------------------+-------------+---------------+------------------+
|355                       |1271         |863            |67.9              |
|356                       |1350         |897            |66.44             |
|357                       |1250         |821            |65.68             |
|354                       |1314         |839            |63.85             |
|364                       |1238         |773            |62.44             |
|363                       |1259         |779            |61.87             |
|362                       |1288         |792            |61.49             |
|2                         |1205         |741            |61.49             |
|345                       |1286         |785            |61.04             |
|361                       |1270         |774            |60.94 

delaysByYearMonth = [feature_flight_day_of_year: int, total_flights: bigint ... 2 more fields]


[feature_flight_day_of_year: int, total_flights: bigint ... 2 more fields]

# feature_flight_week_of_year

In [16]:
import org.apache.spark.sql.functions._

val delaysByYearMonth =
  flightsDF
    .groupBy("feature_flight_week_of_year")
    .agg(
      count(lit(1)).as("total_flights"),
      sum(col("label_is_delayed_15min").cast("long")).as("delayed_flights")
    )
    .withColumn(
      "delay_rate_percent",
      round(col("delayed_flights") / col("total_flights") * 100, 2)
    )
    .orderBy(col("delay_rate_percent").desc)

delaysByYearMonth.show(10, truncate = false)

+---------------------------+-------------+---------------+------------------+
|feature_flight_week_of_year|total_flights|delayed_flights|delay_rate_percent|
+---------------------------+-------------+---------------+------------------+
|51                         |8798         |5377           |61.12             |
|52                         |8464         |5055           |59.72             |
|26                         |8116         |4555           |56.12             |
|28                         |8363         |4654           |55.65             |
|24                         |8575         |4701           |54.82             |
|50                         |7861         |4279           |54.43             |
|29                         |8472         |4577           |54.03             |
|12                         |8213         |4392           |53.48             |
|9                          |8127         |4326           |53.23             |
|30                         |8464         |4495     

delaysByYearMonth = [feature_flight_week_of_year: int, total_flights: bigint ... 2 more fields]


[feature_flight_week_of_year: int, total_flights: bigint ... 2 more fields]

# AvgDelayFeature

In [17]:
import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window

object AvgDelayFeature {

  /**
   * Version optimisée du calcul de l'avg_delay
   * Optimisations:
   * - Window functions au lieu de self-join
   * - Calcul des timestamps une seule fois
   * - Partitionnement intelligent
   * - Cache des données intermédiaires
   */
  def enrichFlightsWithAvgDelay(flightData: DataFrame)(implicit spark: SparkSession): DataFrame = {
    
    import spark.implicits._
    
    // Étape 1 : Préparation une seule fois avec cache
    val flightWithTimestamps = flightData
      .withColumn("utc_departure_timestamp", 
        to_timestamp(
          concat(
            col("UTC_FL_DATE"), 
            lit(" "), 
            lpad(col("feature_utc_departure_hour_rounded"), 2, "0"),
            lit(":00:00")
          )
        )
      )
      .withColumn("6_hour_before_departure", 
        col("utc_departure_timestamp") - expr("INTERVAL 6 HOURS")
      )
      .withColumn("2_hour_before_departure", 
        col("utc_departure_timestamp") - expr("INTERVAL 2 HOURS")
      )
      .repartition(col("ORIGIN_AIRPORT_ID")) // Partitionnement par aéroport
      .cache() // Cache pour éviter recalcul
    
    // Étape 2 : Utiliser Window Functions (plus efficace que self-join)
    // Créer une fenêtre glissante par aéroport
    val windowSpec = Window
      .partitionBy("ORIGIN_AIRPORT_ID")
      .orderBy(col("utc_departure_timestamp").cast("long"))
      .rangeBetween(-6 * 3600, -2 * 3600) // En secondes: -6h à -2h
    
    // Étape 3 : Calculer les statistiques avec window functions
    val enrichedFlights = flightWithTimestamps
      .withColumn("avg_delay", 
        avg(col("label_arr_delay_filled")).over(windowSpec)
      )
      .withColumn("num_previous_flights", 
        count(col("label_arr_delay_filled")).over(windowSpec)
      )
      .withColumn("proportion_delayed_15min", 
        avg(col("label_is_delayed_15min").cast("double")).over(windowSpec)
      )
      .withColumn("stddev_delay", 
        stddev(col("label_arr_delay_filled")).over(windowSpec)
      )
      .withColumn("max_delay", 
        max(col("label_arr_delay_filled")).over(windowSpec)
      )
      .withColumn("min_delay", 
        min(col("label_arr_delay_filled")).over(windowSpec)
      )
      // Remplacer les null par 0 pour les vols sans historique
      .na.fill(0.0, Seq("avg_delay", "proportion_delayed_15min", "stddev_delay", "max_delay", "min_delay"))
      .na.fill(0, Seq("num_previous_flights"))
      // Nettoyer les colonnes intermédiaires
      .drop("6_hour_before_departure", "2_hour_before_departure")
    
    // Unpersist pour libérer la mémoire
    flightWithTimestamps.unpersist()
    
    enrichedFlights
  }
  
  /**
   * Version alternative avec broadcast join si peu d'aéroports
   * Utilise un self-join optimisé avec broadcast
   */
  def enrichFlightsWithAvgDelayBroadcast(flightData: DataFrame)(implicit spark: SparkSession): DataFrame = {
    
    import spark.implicits._
    import org.apache.spark.sql.functions.broadcast
    
    // Préparation des timestamps
    val flightWithTimestamps = flightData
      .withColumn("utc_departure_timestamp", 
        to_timestamp(
          concat(
            col("UTC_FL_DATE"), 
            lit(" "), 
            lpad(col("feature_utc_departure_hour_rounded"), 2, "0"),
            lit(":00:00")
          )
        )
      )
      .withColumn("dep_ts_long", col("utc_departure_timestamp").cast("long"))
      .repartition(200, col("ORIGIN_AIRPORT_ID")) // Augmenter le parallélisme
      .cache()
    
    // Créer un DataFrame agrégé plus petit (candidat pour broadcast)
    val delayStats = flightWithTimestamps
      .groupBy("ORIGIN_AIRPORT_ID", "dep_ts_long")
      .agg(
        avg("label_arr_delay_filled").alias("instant_avg_delay"),
        count("*").alias("flights_at_this_time"),
        avg(col("label_is_delayed_15min").cast("double")).alias("instant_proportion_delayed")
      )
    
    // Self-join optimisé avec range condition
    val enrichedFlights = flightWithTimestamps.as("f1")
      .join(
        delayStats.as("f2"),
        col("f1.ORIGIN_AIRPORT_ID") === col("f2.ORIGIN_AIRPORT_ID") &&
        col("f2.dep_ts_long") >= col("f1.dep_ts_long") - 6 * 3600 &&
        col("f2.dep_ts_long") <= col("f1.dep_ts_long") - 2 * 3600,
        "left"
      )
      .groupBy("f1.*")
      .agg(
        avg("f2.instant_avg_delay").alias("avg_delay"),
        sum("f2.flights_at_this_time").alias("num_previous_flights"),
        avg("f2.instant_proportion_delayed").alias("proportion_delayed_15min")
      )
      .na.fill(0.0, Seq("avg_delay", "proportion_delayed_15min"))
      .na.fill(0, Seq("num_previous_flights"))
      .drop("dep_ts_long")
    
    flightWithTimestamps.unpersist()
    
    enrichedFlights
  }
  
}

defined object AvgDelayFeature


# Flight Delay Metrics

In [30]:
import org.apache.spark.sql.functions._

val delayColumns = Seq(
  "FL_DATE",
  "OP_CARRIER_FL_NUM",
  "ORIGIN_AIRPORT_ID",
  "WEATHER_DELAY",
  "NAS_DELAY",
  "ARR_DELAY_NEW",

  /**"label_is_early",
  "label_is_on_time",
    
  "label_arr_delay_filled",
  "label_nas_delay_filled",
  "label_weather_delay_filled",
    
  "label_weather_delay_was_missing",
  "label_nas_delay_was_missing",
  "label_arr_delay_was_missing",**/
  
  "label_total_weather_nas_delay",

  /**"label_has_nas_delay",
  "label_has_weather_delay",
  "label_has_any_weather_nas_delay",**/
    
  "label_is_delayed_15min",
  /**"label_is_delayed_30min",
  "label_is_delayed_60min",
  "label_is_delayed_45min",
  "label_is_delayed_90min",
  
  "feature_avg_delay",
  "feature_num_previous_flights",
  "feature_stddev_delay",
  "feature_max_delay",
  "feature_min_delay",**/

  "feature_proportion_delayed_15min",    
  /**"feature_proportion_delayed_30min",
  "feature_proportion_delayed_60min",
  "feature_proportion_delayed_45min",
  "feature_proportion_delayed_90min"**/
)

val df = flightsDF
  .select(delayColumns.head, delayColumns.tail: _*)
  .filter(
    col("FL_DATE") === "2013-03-11" &&
    col("OP_CARRIER_FL_NUM") === 3508 &&  
    col("WEATHER_DELAY").isNotNull && col("WEATHER_DELAY") > 0 &&
    col("NAS_DELAY").isNotNull && col("NAS_DELAY") > 0
  )

df.show(5)


+----------+-----------------+-----------------+-------------+---------+-------------+-----------------------------+----------------------+--------------------------------+
|   FL_DATE|OP_CARRIER_FL_NUM|ORIGIN_AIRPORT_ID|WEATHER_DELAY|NAS_DELAY|ARR_DELAY_NEW|label_total_weather_nas_delay|label_is_delayed_15min|feature_proportion_delayed_15min|
+----------+-----------------+-----------------+-------------+---------+-------------+-----------------------------+----------------------+--------------------------------+
|2013-03-11|             3508|            10397|          3.0|     11.0|         18.0|                         14.0|                     1|             0.03508771929824561|
+----------+-----------------+-----------------+-------------+---------+-------------+-----------------------------+----------------------+--------------------------------+



delayColumns = List(FL_DATE, OP_CARRIER_FL_NUM, ORIGIN_AIRPORT_ID, WEATHER_DELAY, NAS_DELAY, ARR_DELAY_NEW, label_total_weather_nas_delay, label_is_delayed_15min, feature_proportion_delayed_15min)
df = [FL_DATE: date, OP_CARRIER_FL_NUM: int ... 7 more fields]


[FL_DATE: date, OP_CARRIER_FL_NUM: int ... 7 more fields]